* Base de datos: 2007-2020
* Tipo corrupción: Intensa

## 1. Cargar datos

In [1]:
import warnings
warnings.filterwarnings( 'ignore' )

In [2]:
import pandas as pd
import numpy as np
import pickle
import joblib
from glob import glob

In [4]:
path      = r'..\..\..\input\prediction_data\ejecucion_6\b0_ci_siaf\preprocessed_data\*.csv'
path_list = glob( path )
dfs       = []

for i, base in enumerate( path_list ):
    df = pd.read_csv( base, dtype = { 'ubigeo': str, 'year': str } )
    dfs.append( df )

## 2. Hacer predicciones

In [5]:
optimal_model = joblib.load( r'..\..\..\output\ejecucion_6\models\base0\model_ci_Random Forest Classifier_nro.joblib' )

In [7]:
for i, df in enumerate( dfs ):
    
    ubigeo_column                        = dfs[ i ][ 'ubigeo' ].copy()
    year_column                          = dfs[ i ][ 'year' ].copy()
    dfs[ i ]                             = dfs[ i ].drop( [ 'ubigeo', 'year' ], axis = 1 )
    
    y_pred_class_o                       = optimal_model.predict( dfs[ i ] )
    y_pred_prob_o                        = optimal_model.predict_proba( dfs[ i ] )[ :, 1 ]  
    dfs[ i ][ 'prob_ci' ]                = y_pred_prob_o
    dfs[ i ][ 'class_ci_default' ]       = y_pred_class_o
    dfs[ i ][ 'class_ci_calculated_50' ] = dfs[ i ][ 'prob_ci' ].apply( lambda x: 1 if x > 0.5 else 0 )
    dfs[ i ][ 'class_ci_calculated_60' ] = dfs[ i ][ 'prob_ci' ].apply( lambda x: 1 if x > 0.6 else 0 )
    dfs[ i ][ 'class_ci_calculated_70' ] = dfs[ i ][ 'prob_ci' ].apply( lambda x: 1 if x > 0.7 else 0 )
    dfs[ i ][ 'class_ci_100' ]           = ( dfs[ i ][ 'prob_ci' ].rank( ascending = False, method = 'first' ) <= 100 ).astype( int )
    
    dfs[ i ][ 'ubigeo' ]                = ubigeo_column
    dfs[ i ][ 'year' ]                  = year_column
    
    dfs[ i ]                            = dfs[ i ][ [ 'ubigeo', 'year', 'prob_ci', 'class_ci_default', 
                                                      'class_ci_calculated_50', 'class_ci_calculated_60',
                                                      'class_ci_calculated_70', 'class_ci_100' ] ]

In [8]:
complete_base = pd.concat( dfs, axis = 0 )

## 3. Exportar datos

In [9]:
complete_base.to_csv( r'..\..\..\input\prediction_data\ejecucion_6\b0_ci_siaf\predicted_data\base0_complete.csv', index = False )

In [15]:
# years_list = np.arange( 2009, 2021, 1 )

# for i, year in enumerate( years_list ):
#     dfs[ i ].to_csv( f'../../../input/prediction_data/ejecucion_6/b0_ci_siaf/predicted_data/base0_{ year }.csv', index = False )